# Preprocess APO fluxes from CarboScope inversion

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import cftime

import pandas as pd
import xarray as xr
import intake

import config
import regrid_tools
import util

## Read APO flux data

In [3]:
cat = intake.open_catalog("catalogs/fluxes-catalog-web.yml")
cat

fluxes-catalog-web:
  args:
    path: catalogs/fluxes-catalog-web.yml
  description: Flux datasets
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


In [4]:
version = "apo99_v2020"

### Fix `storage_options` to increase `timeout` 

In [5]:
import aiohttp
timeout = aiohttp.ClientTimeout(total=86400)
storage_options=dict(**cat[f"fgapo_CarbonScope_{version}"].storage_options)
storage_options.update(
    dict(
        http=dict(client_kwargs={'timeout': timeout}),
    )
)
storage_options

{'simplecache': {'same_names': True,
  'cache_storage': '/glade/work/mclong/sno-analysis/intake-cache/test'},
 'http': {'client_kwargs': {'timeout': ClientTimeout(total=86400, connect=None, sock_read=None, sock_connect=None)}}}

In [ ]:
ds = cat[f"fgapo_CarbonScope_{version}"](storage_options=storage_options).to_dask().compute()            
ds

In [ ]:
ds.apoflux_ocean.attrs

In [ ]:
ds.apoflux_ocean.isel(mtime=0).plot()

In [ ]:
grid = util.generate_latlon_grid(nx=ds.lon.size, ny=ds.lat.size, lon0=-180.)
np.testing.assert_array_almost_equal(grid.lon.values, ds.lon.values)
np.testing.assert_array_almost_equal(grid.lat.values, ds.lat.values)

src_grid = regrid_tools.grid("latlon", nx=ds.lon.size, ny=ds.lat.size, lon0=-180.)
src_grid

In [ ]:
dt = ds.dt.values[0]
np.testing.assert_equal(ds.dt.values, dt)

In [ ]:
Tmolyr_to_mols = 1e12 / 365. / 86400.
flux_variables = dict(
    fgapo_ocn="apoflux_ocean",
    fgapo_lnd="apoflux_land",
    fgapo_fff="apoflux_subt",
    fgapo_exc="apoflux_excl",
)
dso_src = ds[["mtime", "lat", "lon"]]
for var_out, var_in in flux_variables.items():    
    dso_src[var_out] = ds[var_in] / ds.dxyp * Tmolyr_to_mols
    dso_src[var_out].attrs = ds[var_in].attrs
    dso_src[var_out].attrs["units"] = "mol/m^2/s"
    del dso_src[var_out].attrs["cell_measures"]
    dso_src[var_out].encoding = ds[var_in].encoding
    
dso_src = dso_src.rename({"mtime": "time"})
dso_src.time.attrs = ds.mtime.attrs
dso_src.time.encoding["_FillValue"] = None
dso_src

In [ ]:
dst_grid = regrid_tools.grid("latlon", **config.config_dict["flux-dst-grid-kwargs"])
dst_grid

In [ ]:
regrid_obj = regrid_tools.regridder(src_grid, dst_grid, method="conserve", clobber=True)
regrid_obj

In [ ]:
dso = util.generate_latlon_grid(**config.config_dict["flux-dst-grid-kwargs"])[["area"]]

dso_dst_data = regrid_obj(dso_src)
dso["time"] = dso_src.time

for v in dso_dst_data.data_vars:
    dso[v] = dso_dst_data[v]
    dso[v].encoding = dso_src[v].encoding
    
time = cftime.num2date(dso.time, units=dso.time.attrs["units"])    
dso["date"] = xr.DataArray(
    [util.yyyymmdd(d.year, d.month, d.day) for d in time],
    dims=("time"),
    attrs={"long_name": "date", "units": "YYYYMMDD"},
    coords={"time": dso.time},
)
dso

In [ ]:
file_out = f"{config.flux_product_dir}/CarboScope.{version}.fgapo.nc"
dso.to_netcdf(file_out)

In [ ]:
dso.info()